# ETL CODE
---

## Extract data from datafiles and API


In [3]:
import requests
import os
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, TimestampType, FloatType
from pyspark.sql.functions import col, initcap, lower, concat_ws, concat, lit, substring, when, lpad

spark = SparkSession.builder.appName("credit_card").getOrCreate()

Create the schemas for the data

In [4]:
schema_branches = StructType([
                            StructField("BRANCH_CODE", IntegerType(), False),
                            StructField("BRANCH_NAME", StringType()), 
                            StructField("BRANCH_STREET", StringType()),
                            StructField("BRANCH_CITY", StringType()),
                            StructField("BRANCH_STATE", StringType()),
                            StructField("BRANCH_ZIP", StringType()),
                            StructField("BRANCH_PHONE", StringType()),
                            StructField("LAST_UPDATED", TimestampType())
])

schema_customers = StructType([
                            StructField("FIRST_NAME", StringType()),
                            StructField("MIDDLE_NAME", StringType()),
                            StructField("LAST_NAME", StringType()),
                            StructField("SSN", IntegerType(), False),
                            StructField("CREDIT_CARD_NO", StringType()),
                            StructField("APT_NO", StringType()),
                            StructField("STREET_NAME", StringType()),
                            StructField("CUST_CITY", StringType()),
                            StructField("CUST_STATE", StringType()),
                            StructField("CUST_COUNTRY", StringType()),
                            StructField("CUST_ZIP", StringType()),
                            StructField("CUST_PHONE", StringType()),
                            StructField("CUST_EMAIL", StringType()),
                            StructField("LAST_UPDATED", TimestampType())
])

schema_cc_transactions = StructType([
                            StructField("TRANSACTION_ID", IntegerType(), False),
                            StructField("DAY", IntegerType()),
                            StructField("MONTH", IntegerType()),
                            StructField("YEAR", IntegerType()),
                            StructField("CREDIT_CARD_NO", StringType()),
                            StructField("CUST_SSN", StringType()),
                            StructField("BRANCH_CODE", IntegerType()),
                            StructField("TRANSACTION_TYPE", StringType()),
                            StructField("TRANSACTION_VALUE", FloatType())
])

schema_loans = StructType([
                            StructField("Application_ID", StringType()),
                            StructField("Gender", StringType()),
                            StructField("Married", StringType()),
                            StructField("Dependents", StringType()),
                            StructField("Education", StringType()),
                            StructField("Self_Employed", StringType()),
                            StructField("Credit_History", StringType()),
                            StructField("Property_Area", StringType()),
                            StructField("Income", StringType()),
                            StructField("Application_Status", StringType())
])


Importing in the all data from files and API (I know too soon, I'll cover this later on)

In [9]:
branches_df = spark.read.json("../data_files/cdw_sapp_branch.json", schema=schema_branches)
branches_df.show(5)

customers_df = spark.read.json("../data_files/cdw_sapp_customer.json", schema=schema_customers)
customers_df.show(5)

cc_transactions_df = spark.read.json("../data_files/cdw_sapp_credit.json", schema=schema_cc_transactions)
cc_transactions_df.show(5)

resp = requests.get("https://raw.githubusercontent.com/platformps/LoanDataset/main/loan_data.json")
loan_df = spark.createDataFrame(resp.json(), schema=schema_loans)
loan_df.show(5)



+-----------+------------+-----------------+-----------------+------------+----------+------------+-------------------+
|BRANCH_CODE| BRANCH_NAME|    BRANCH_STREET|      BRANCH_CITY|BRANCH_STATE|BRANCH_ZIP|BRANCH_PHONE|       LAST_UPDATED|
+-----------+------------+-----------------+-----------------+------------+----------+------------+-------------------+
|          1|Example Bank|     Bridle Court|        Lakeville|          MN|     55044|  1234565276|2018-04-18 15:51:47|
|        999|   TEST Bank|      Night Court|         TreeCity|          MM|      null|  1234567890|2018-04-18 15:51:47|
|          2|Example Bank|Washington Street|          Huntley|          IL|     60142|  1234618993|2018-04-18 15:51:47|
|          3|Example Bank|    Warren Street|SouthRichmondHill|          NY|     11419|  1234985926|2018-04-18 15:51:47|
|          4|Example Bank| Cleveland Street|       Middleburg|          FL|     32068|  1234663064|2018-04-18 15:51:47|
+-----------+------------+--------------

KeyboardInterrupt: 

+-----------+------------+-----------------+-----------------+------------+----------+------------+-------------------+
|BRANCH_CODE| BRANCH_NAME|    BRANCH_STREET|      BRANCH_CITY|BRANCH_STATE|BRANCH_ZIP|BRANCH_PHONE|       LAST_UPDATED|
+-----------+------------+-----------------+-----------------+------------+----------+------------+-------------------+
|          1|Example Bank|     Bridle Court|        Lakeville|          MN|     55044|  1234565276|2018-04-18 15:51:47|
|        999|   TEST Bank|      Night Court|         TreeCity|          MM|      null|  1234567890|2018-04-18 15:51:47|
|          2|Example Bank|Washington Street|          Huntley|          IL|     60142|  1234618993|2018-04-18 15:51:47|
|          3|Example Bank|    Warren Street|SouthRichmondHill|          NY|     11419|  1234985926|2018-04-18 15:51:47|
|          4|Example Bank| Cleveland Street|       Middleburg|          FL|     32068|  1234663064|2018-04-18 15:51:47|
+-----------+------------+--------------

## Transformation with PySpark


Transforming customer data

In [10]:
# Convert the first_name to Title case
print("Convert the first_name to Title case")
customers_df = customers_df.withColumn("FIRST_NAME", initcap(col("FIRST_NAME")))
customers_df.show(3)

# Convert the middle name to lower case
print("Convert the middle name in lower case")
customers_df = customers_df.withColumn("MIDDLE_NAME", lower(col("MIDDLE_NAME")))
customers_df.show(3)

# Convert the Last Name to Title Case
print("Convert the Last Name in Title Case")
customers_df = customers_df.withColumn("LAST_NAME", initcap(col("LAST_NAME")))
customers_df.show(3)

# Concatenate Apartment no and Street name of customer's Residence with comma as a seperator (Street, Apartment)
print("Concatenate Apartment no and Street name of customer's Residence with comma as a seperator (Street, Apartment)")
customers_df2 = customers_df.withColumn("STREET_NAME", concat_ws(', ',col("STREET_NAME"),col("APT_NO")))
customers_df2 = customers_df2.withColumnRenamed("STREET_NAME", "FULL_STREET_ADDRESS")
customers_df2 = customers_df2.drop("APT_NO")
customers_df2.show(3)

# Change the format of phone number to (XXX)XXX-XXXX....the data ONLY has 7 digits NOT 10, padding the area code with 555
print("Change the format of the phone number to (XXX)XXX-XXXX")
customers_df3 = customers_df2.withColumn("CUST_PHONE", 
                                            concat(
                                                    lit("(555)"),
                                                    substring(col("CUST_PHONE"), 1, 3),
                                                    lit("-"),
                                                    substring(col("CUST_PHONE"), 4, 4)
                                                    ))
customers_df3.show(3)


# Rearranging columns
print("Rearranging columns")
customers_df3 = customers_df3.select(
                                    "SSN" ,
                                    "FIRST_NAME" ,
                                    "MIDDLE_NAME" ,
                                    "LAST_NAME" ,
                                    "CREDIT_CARD_NO" ,
                                    "FULL_STREET_ADDRESS",
                                    "CUST_CITY" ,
                                    "CUST_STATE" ,
                                    "CUST_COUNTRY" ,
                                    "CUST_ZIP" ,
                                    "CUST_PHONE" ,
                                    "CUST_EMAIL" ,
                                    "LAST_UPDATED"
)
customers_df3.show(3)


Convert the first_name to Title case
+----------+-----------+---------+---------+----------------+------+-----------------+------------+----------+-------------+--------+----------+-------------------+-------------------+
|FIRST_NAME|MIDDLE_NAME|LAST_NAME|      SSN|  CREDIT_CARD_NO|APT_NO|      STREET_NAME|   CUST_CITY|CUST_STATE| CUST_COUNTRY|CUST_ZIP|CUST_PHONE|         CUST_EMAIL|       LAST_UPDATED|
+----------+-----------+---------+---------+----------------+------+-----------------+------------+----------+-------------+--------+----------+-------------------+-------------------+
|      Alec|         Wm|   Hooper|123456100|4210653310061055|   656|Main Street North|     Natchez|        MS|United States|   39120|   1237818|AHooper@example.com|2018-04-21 11:49:02|
|      Etta|    Brendan|   Holman|123453023|4210653310102868|   829|    Redwood Drive|Wethersfield|        CT|United States|   06109|   1238933|EHolman@example.com|2018-04-21 11:49:02|
|    Wilber|   Ezequiel|   Dunham|1234

Transforming branch data

In [11]:
# If the BRANCH_ZIP source value is null load default (99999) value else Direct move
branches_df = branches_df.withColumn("BRANCH_ZIP", when(col("BRANCH_ZIP").isNull(), 99999).otherwise(col("BRANCH_ZIP")))
branches_df.show(5)


# Change the format of phone number to (XXX)XXX-XXXX
branches_df2 = branches_df.withColumn("BRANCH_PHONE", 
                                      concat(
                                            lit("("),
                                            substring(col("BRANCH_PHONE"), 0, 3),
                                            lit(")"),
                                            substring(col("BRANCH_PHONE"), 4, 3),
                                            lit("-"),
                                            substring(col("BRANCH_PHONE"), 7, 4)
                                             ))
branches_df2.show(3)


+-----------+------------+-----------------+-----------------+------------+----------+------------+-------------------+
|BRANCH_CODE| BRANCH_NAME|    BRANCH_STREET|      BRANCH_CITY|BRANCH_STATE|BRANCH_ZIP|BRANCH_PHONE|       LAST_UPDATED|
+-----------+------------+-----------------+-----------------+------------+----------+------------+-------------------+
|          1|Example Bank|     Bridle Court|        Lakeville|          MN|     55044|  1234565276|2018-04-18 15:51:47|
|        999|   TEST Bank|      Night Court|         TreeCity|          MM|     99999|  1234567890|2018-04-18 15:51:47|
|          2|Example Bank|Washington Street|          Huntley|          IL|     60142|  1234618993|2018-04-18 15:51:47|
|          3|Example Bank|    Warren Street|SouthRichmondHill|          NY|     11419|  1234985926|2018-04-18 15:51:47|
|          4|Example Bank| Cleveland Street|       Middleburg|          FL|     32068|  1234663064|2018-04-18 15:51:47|
+-----------+------------+--------------

Transforming credit card transaction data

In [12]:
# Convert DAY, MONTH, and YEAR into a TIMEID (YYYYMMDD)
cc_transactions_df2 = cc_transactions_df.withColumn("DAY", concat("YEAR", lpad("MONTH", 2, '0'), lpad("DAY", 2, '0')))\
                                        .withColumnRenamed("DAY","TIMEID")\
                                        .drop("MONTH")\
                                        .drop("YEAR")
cc_transactions_df2.show(3)

+--------------+--------+----------------+---------+-----------+----------------+-----------------+
|TRANSACTION_ID|  TIMEID|  CREDIT_CARD_NO| CUST_SSN|BRANCH_CODE|TRANSACTION_TYPE|TRANSACTION_VALUE|
+--------------+--------+----------------+---------+-----------+----------------+-----------------+
|             1|20180214|4210653349028689|123459988|        114|       Education|             78.9|
|             2|20180320|4210653349028689|123459988|         35|   Entertainment|            14.24|
|             3|20180708|4210653349028689|123459988|        160|         Grocery|             56.7|
+--------------+--------+----------------+---------+-----------+----------------+-----------------+
only showing top 3 rows



## Load into database

First I updated the create_tables.sql script to reflect the transformations made above

In [18]:
#############################
##  LOCAL DATABASE SERVER  ##
#############################

#conn_url = "jdbc:mysql://mikey.helioho.st:3306/michaelwschmidt_creditcard_capstone",
#table = "cdw_sapp_customer",
#conn_mode = "append"
with open('../.secrets', 'r') as f:
    CREDITCARD_CAPSTONE_PASSWORD = f.read()

conn_prop = {
            "driver": "com.mysql.cj.jdbc.Driver",
            "url": "jdbc:mysql://localhost:3306/creditcard_capstone",
            "user": "root",
            "password": "password",
            "header": "false"
}

table_name = "cdw_sapp_customer"
customers_df3.write.format("jdbc").mode("append")\
                .option("driver", conn_prop["driver"])\
                .option("url", conn_prop["url"])\
                .option("user", conn_prop["user"])\
                .option("dbtable", table_name)\
                .option("truncate","true")\
                .option("password", conn_prop["password"])\
                .option("header", conn_prop["header"])\
                .save()

table_name = "cdw_sapp_branch"
branches_df2.write.format("jdbc").mode("append")\
                .option("driver", conn_prop["driver"])\
                .option("url", conn_prop["url"])\
                .option("user", conn_prop["user"])\
                .option("dbtable", table_name)\
                .option("truncate","true")\
                .option("password", conn_prop["password"])\
                .option("header", conn_prop["header"])\
                .save()

table_name = "cdw_sapp_credit_card"
cc_transactions_df2.write.format("jdbc").mode("append")\
                .option("driver", conn_prop["driver"])\
                .option("url", conn_prop["url"])\
                .option("user", conn_prop["user"])\
                .option("dbtable", table_name)\
                .option("truncate","true")\
                .option("password", conn_prop["password"])\
                .option("header", conn_prop["header"])\
                .save()


In [ ]:
#######################################################
##  ONLINE SERVER  MAKE SURE THE HOST IS AVAILABLE  ##
#######################################################
'''
conn_url = "jdbc:mysql://mikey.helioho.st:3306/michaelwschmidt_creditcard_capstone",
table = "cdw_sapp_customer",
conn_mode = "append"
with open('../.secrets', 'r') as f:
    CREDITCARD_CAPSTONE_PASSWORD = f.read()

conn_prop = {
            "driver": "com.mysql.cj.jdbc.Driver",
            "user": "michaelwschmidt_cc_capstone",
            "password": CREDITCARD_CAPSTONE_PASSWORD,
            "dbtable": "michaelwschmidt_creditcard_capstone.cdw_sapp_customer",
            "header": False
}


customers_df3.write.format("jdbc").mode("append")\
                .option("driver", "com.mysql.cj.jdbc.Driver")\
                .option("url", "jdbc:mysql://tommy2.heliohost.org:3306/michaelwschmidt_creditcard_capstone")\
                .option("user", "michaelwschmidt_cc_capstone")\
                .option("dbtable","cdw_sapp_customer")\
                .option("password", CREDITCARD_CAPSTONE_PASSWORD)\
                .option("header", "false")\
                .save()

branches_df2.write.format("jdbc").mode("append")\
                .option("driver", "com.mysql.cj.jdbc.Driver")\
                .option("url", "jdbc:mysql://mikey.helioho.st:3306/michaelwschmidt_creditcard_capstone")\
                .option("user", "michaelwschmidt_cc_capstone")\
                .option("dbtable","cdw_sapp_branch")\
                .option("password", CREDITCARD_CAPSTONE_PASSWORD)\
                .option("header", "false")\
                .save()


cc_transactions_df2.write.format("jdbc").mode("append")\
                .option("driver", "com.mysql.cj.jdbc.Driver")\
                .option("url", "jdbc:mysql://mikey.helioho.st:3306/michaelwschmidt_creditcard_capstone")\
                .option("user", "michaelwschmidt_cc_capstone")\
                .option("dbtable","cdw_sapp_credit_card")\
                .option("password", CREDITCARD_CAPSTONE_PASSWORD)\
                .option("header", "false")\
                .save()
'''